In [6]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
import time

In [7]:
def add_pruning_data(pruning_amounts, pre_accuracy, post_accuracy, comp_time, model_size, step=1, window_size=7, poly_order=6, noise=1.0):
    df = pd.DataFrame({'pruning': pruning_amounts, 'pre_accuracy': pre_accuracy,'accuracy': post_accuracy, 'comp_time': comp_time, 'model_size': model_size})
    
    new_pruning_amounts = np.arange(0, np.max(pruning_amounts) + step, step)
    df_interp = df.set_index('pruning').reindex(new_pruning_amounts).interpolate(method='linear')
    
    # Apply Savitzky-Golay filter for smoothing
    def smooth_series(series):
        return savgol_filter(series, window_size, poly_order, mode='nearest') if len(series) > window_size else series
    
    df_interp['pre_accuracy_smooth'] = smooth_series(df_interp['pre_accuracy'].values)
    df_interp['post_accuracy_smooth'] = smooth_series(df_interp['accuracy'].values)
    df_interp['comp_time_smooth'] = smooth_series(df_interp['comp_time'].values)
    df_interp['model_size_smooth'] = smooth_series(df_interp['model_size'].values)
    
    # Add noise if specified
    if noise != 0.0:
        new_pre_acc = smooth_series(df_interp['pre_accuracy_smooth'].values) + np.random.logistic(scale=noise, size=len(df_interp))
        new_post_acc = smooth_series(df_interp['post_accuracy_smooth'].values) + np.random.logistic(scale=noise, size=len(df_interp))
        new_comp_time = smooth_series(df_interp['comp_time_smooth'].values) + np.random.logistic(scale=noise, size=len(df_interp))
        new_model_size = smooth_series(df_interp['model_size_smooth'].values) + np.random.logistic(scale=noise, size=len(df_interp))
        return new_pruning_amounts, new_pre_acc, new_post_acc, new_comp_time, new_model_size
    else:
        return new_pruning_amounts, df_interp['pre_accuracy_smooth'].values, df_interp['post_accuracy_smooth'].values, df_interp['comp_time_smooth'].values, df_interp['model_size_smooth'].values


In [8]:
pruning_amounts = []
pre_accuracies = []
post_accuracies = []
comp_time = []
model_size = []
with open('Sheet 1-VGG16_results.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        pruning_amounts.append(float(row[0]))
        pre_accuracies.append(float(row[2]))
        post_accuracies.append(float(row[4]))
        comp_time.append(float(row[5]))
        model_size.append(float(row[6]))
pruning_amounts = np.array(pruning_amounts)
pre_accuracies = np.array(pre_accuracies)
post_accuracies = np.array(post_accuracies)
comp_time = np.array(comp_time)
model_size = np.array(model_size)

In [9]:
new_pruning_amounts, new_pre_acc, new_post_acc, new_comp_time, new_model_size = add_pruning_data(pruning_amounts, pre_accuracies, post_accuracies, comp_time, model_size, step=1, window_size=7, poly_order=6, noise=0.0)

In [10]:
def save_to_csv(new_pruning_amounts, new_pre_acc, new_post_acc, new_comp_time, new_model_size):
    with open('Sheet 1-VGG16_results_interpolated.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pruning', 'pre_accuracy', 'post_accuracy', 'comp_time', 'model_size'])
        for i in range(len(new_pruning_amounts)):
            writer.writerow([new_pruning_amounts[i], new_pre_acc[i], new_post_acc[i], new_comp_time[i], new_model_size[i]])
save_to_csv(new_pruning_amounts, new_pre_acc, new_post_acc, new_comp_time, new_model_size)